# Langfuse 로 Amazon Bedrock 시작하기

이 노트북은 [Langfuse](https://langfuse.com/) 를 사용하여 Amazonb Bedrock 의 LLM 을 사용하는 방법을 
가이드하는 노트북 입니다.

이 노트북을 사용하기 전에 아래의 참조 자료를 확인 해보세요. 

#### 참조 자료: 

* LLM 앱 디버깅 툴, Langfuse를 Amazon ECS에 배포하는 방법, [Youtube Link](https://www.youtube.com/watch?v=rrPQcWq5pe8&t=5s)
* Hosting Langfuse V3 on Amazon ECS with Fargate using CDK Python, [Git Repo](https://github.com/aws-samples/deploy-langfuse-on-ecs-with-fargate/tree/main/langfuse-v3)
* Amazon ECS와 AWS Fargate를 사용하여 AWS CDK Python으로 Langfuse 호스팅하기, [Blog Link](https://aws.amazon.com/ko/blogs/tech/hosting-langfuse-with-aws-cdk-python-using-amazon-ecs-and-aws-fargate/)


---

## 1. 선수 사항: 
### 1.1 Langfuse 를 AWS 인프라에 호스팅 
- 아래의 설치 가이드에 따라 Langfuse 를 먼저 설치 하세요.
    - [설치 가이드: Langfuse 를 AWS 인프라에 호스팅 ](../../setup/LAGNFUSE_README.md)

### 1.2. 선수 사항: 콘다 가상 환경 생성
- [setup](../../setup/README.md) 의 가이드에 따라 실행하여, 가상 환경인 langgraph 생성
- 이후 모든 노트북의 커널은 langgraph 를 사용합니다.

### 1.3. Key 정보를 저장하는 env 파일 생성
-  ../../.env 파일을 생성하고 아래의 내용을 작성
    ```
    LANGFUSE_SECRET_KEY=<secret key>
    LANGFUSE_PUBLIC_KEY=<public key>
    LANGFUSE_HOST=<host url>
    ```


## 2. .env 파일을 통하여 key 정보 불러오기

In [1]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수 로드
load_dotenv("../../.env")



True

## 3. langfuse_handler 핸들러 작성하기

In [3]:
from langfuse.callback import CallbackHandler
import os

langfuse_handler = CallbackHandler(
    public_key=os.environ.get('LANGFUSE_PUBLIC_KEY'),
    secret_key=os.environ.get('LANGFUSE_SECRET_KEY'),
    host=os.environ.get('LANGFUSE_HOST'),
)

## 4. langfuse endpoint 에 인증 해보기

In [4]:
# connection test
langfuse_handler.auth_check()

True

## 5. LangCahin 모델 오브젝트 생성 및 실행

In [5]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs=dict(temperature=0),
    # other params...
)

In [6]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Korean. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages, config={"callbacks": [langfuse_handler]})
ai_msg



AIMessage(content='프로그래밍을 좋아합니다.', additional_kwargs={'usage': {'prompt_tokens': 29, 'completion_tokens': 19, 'total_tokens': 48}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, response_metadata={'usage': {'prompt_tokens': 29, 'completion_tokens': 19, 'total_tokens': 48}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, id='run-1acb4d91-48ce-418b-bef8-a8da9d74e319-0', usage_metadata={'input_tokens': 29, 'output_tokens': 19, 'total_tokens': 48})

In [7]:
print(ai_msg.content)

프로그래밍을 좋아합니다.


## 6. LangChain 모델 Chaining 해보기

In [8]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "korean",
        "input": "I love programming.",
    },
    config={"callbacks": [langfuse_handler]}
)

AIMessage(content='프로그래밍을 좋아합니다.', additional_kwargs={'usage': {'prompt_tokens': 23, 'completion_tokens': 19, 'total_tokens': 42}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, response_metadata={'usage': {'prompt_tokens': 23, 'completion_tokens': 19, 'total_tokens': 42}, 'stop_reason': 'end_turn', 'thinking': {}, 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, id='run-3ce27afd-04ad-4025-8330-7836df81f24e-0', usage_metadata={'input_tokens': 23, 'output_tokens': 19, 'total_tokens': 42})

### 6.1.LangFuse 추적 내용 보기
로컬에 호스팅된 LangFuse 에 로그인을 하면 아래와 같이 기록이 남습니다.
- LLM 호출건수 추적 기록 남음
    - ![lang_run_result_01.png](img/lang_run_result_01.png)
- 아래는 위에서 chain.invoke() 를 실행시에 chain = prompt | llm 여기서 prompt 가 실행된 것을 추적 합니다.    
    - ![lang_chain_template.png](img/lang_chain_template.png)  
- 아래는 위에서 chain.invoke() 를 실행시에 chain = prompt | llm 여기서 lim 이 실행된 것을 추적 합니다.        
    - ![lang_chain_chatbedrock.png](img/lang_chain_chatbedrock.png)  



## 7. Bedrock Converse API 사용해보기

In [9]:
from langchain_aws import ChatBedrockConverse

llm = ChatBedrockConverse(
    model="anthropic.claude-3-sonnet-20240229-v1:0",
    temperature=0,
    max_tokens=None,
    # other params...
)

llm.invoke(messages, config={"callbacks": [langfuse_handler]})

AIMessage(content='프로그래밍을 좋아합니다.', additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '4bf5cd70-2046-412a-a2f1-ce2f1155d4fb', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 12 Mar 2025 07:43:25 GMT', 'content-type': 'application/json', 'content-length': '216', 'connection': 'keep-alive', 'x-amzn-requestid': '4bf5cd70-2046-412a-a2f1-ce2f1155d4fb'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [464]}}, id='run-e01757bd-8b2d-456d-ae30-b42a9a2dd481-0', usage_metadata={'input_tokens': 29, 'output_tokens': 19, 'total_tokens': 48})

In [10]:
for chunk in llm.stream(messages, config={"callbacks": [langfuse_handler]}):
    print(chunk)

content=[] additional_kwargs={} response_metadata={} id='run-7f8b34c8-dd64-4320-963c-4cb40f22477f'
content=[{'type': 'text', 'text': '프', 'index': 0}] additional_kwargs={} response_metadata={} id='run-7f8b34c8-dd64-4320-963c-4cb40f22477f'
content=[{'type': 'text', 'text': '로', 'index': 0}] additional_kwargs={} response_metadata={} id='run-7f8b34c8-dd64-4320-963c-4cb40f22477f'
content=[{'type': 'text', 'text': '그', 'index': 0}] additional_kwargs={} response_metadata={} id='run-7f8b34c8-dd64-4320-963c-4cb40f22477f'
content=[{'type': 'text', 'text': '래', 'index': 0}] additional_kwargs={} response_metadata={} id='run-7f8b34c8-dd64-4320-963c-4cb40f22477f'
content=[{'type': 'text', 'text': '밍', 'index': 0}] additional_kwargs={} response_metadata={} id='run-7f8b34c8-dd64-4320-963c-4cb40f22477f'
content=[{'type': 'text', 'text': '을', 'index': 0}] additional_kwargs={} response_metadata={} id='run-7f8b34c8-dd64-4320-963c-4cb40f22477f'
content=[{'type': 'text', 'text': ' ', 'index': 0}] additiona

## 8. LangChain 의 StrOutputParser 사용해보기

In [11]:
from langchain_core.output_parsers import StrOutputParser

chain = llm | StrOutputParser()

for chunk in chain.stream(messages, config={"callbacks": [langfuse_handler]}):
    print(chunk, end="|")

|프|로|그|래|밍|을| |좋|아|합|니|다|.||||